In [ ]:
# @title 🛠️ Rex2Fusion_Linklist
# @markdown Enter the Start Number and click **Play** (▶️).

import time
import pandas as pd
import re
import os
from google.colab import files
import io

# @markdown ### Settings
Linklist_Start_Number = 50 # @param {type:"integer"}

print("Waiting for file upload...")

# 1. Upload
uploaded = files.upload()

if not uploaded:
    print("❌ No file uploaded.")
else:
    start_time = time.time()
    filename = next(iter(uploaded))
    print(f"\nProcessing: {filename}...")

    # --- PART A: ROBUST READING ---
    raw_data = uploaded[filename]
    try:
        content = raw_data.decode('utf-8-sig', errors='ignore')
    except:
        content = raw_data.decode('latin-1', errors='ignore')

    content = content.replace('\x00', '')
    lines = content.splitlines()

    # --- PART B: PARSE DATA ---
    collections = []     # For File 1
    unit_map = {}        # For File 1 (Linked Units)

    # Structure for File 2: {id: 50, name: "Hood", units: ["Code1", "Code2"]}
    parsed_collections = []

    processing_id = Linklist_Start_Number

    # Fast Regex Patterns
    name_pattern = re.compile(r"^#([^;]+)")
    code_pattern = re.compile(r"#([^\[;#]+)\[([\d\.,]+)\]")

    for line in lines:
        line = line.strip()
        if not line: continue

        # 1. Collection Line
        if line.startswith("#"):
            # Name
            name_match = name_pattern.search(line)
            coll_name = name_match.group(1).strip() if name_match else line.split(";")[0].replace("#", "").strip()

            # Codes
            found_codes = code_pattern.findall(line)
            coll_units = []

            for code, qty_str in found_codes:
                qty_str = qty_str.replace(",", ".")
                try:
                    qty = float(qty_str)
                    loop_count = int(qty) if qty.is_integer() else 1
                except:
                    loop_count = 1

                # Expand rows for File 1
                for _ in range(loop_count):
                    collections.append({
                        "Linklist ID": processing_id,
                        "Linklist Name": coll_name,
                        "Linklist Units": code.strip(),
                        "Number of Units": "[1.000]",
                        "Series ID": 0
                    })
                    # Add to list for File 2 (Linklist Import)
                    coll_units.append(code.strip())

            # Save structured data for File 2
            parsed_collections.append({
                "id": processing_id,
                "name": coll_name,
                "units": coll_units
            })

            processing_id += 1

        # 2. Unit Line (for File 1 Linked Lists)
        elif line.startswith(";#"):
            unit_name = line.replace(";", "").replace("#", "").strip()
            if unit_name:
                active_id = processing_id - 1
                if unit_name not in unit_map: unit_map[unit_name] = []
                unit_map[unit_name].append(active_id)

    # --- PART C: GENERATE FILE 1 (Main Converted File) ---
    print("-> Generating File 1...")
    df1 = pd.DataFrame(collections)
    req_cols = ["Linklist ID", "Linklist Name", "Linklist Units", "Number of Units", "Series ID"]
    for c in req_cols:
        if c not in df1.columns: df1[c] = ""

    unit_data_list = []
    for u_name, id_list in unit_map.items():
        link_str = "<"
        for idx, linked_id in enumerate(id_list):
            link_str += f"{idx+1}~1~{linked_id}|"
        link_str += ">"
        unit_data_list.append({"Unit Name": u_name, "Linked Lists": link_str})

    df1_units = pd.DataFrame(unit_data_list)
    df1_final = pd.concat([df1.reset_index(drop=True), df1_units.reset_index(drop=True)], axis=1)

    final_cols_1 = ["Linklist ID", "Linklist Name", "Linklist Units", "Number of Units", "Series ID", "Unit Name", "Linked Lists"]
    for c in final_cols_1:
        if c not in df1_final.columns: df1_final[c] = ""
    df1_final = df1_final[final_cols_1].fillna("")

    # --- PART D: GENERATE FILE 2 (Import_Linklist) ---
    print("-> Generating File 2...")
    tab2_rows = []

    # Static Values Reference (Row 2 of your sample)
    static_vals = {
        "Add OR Delete": "A",
        "Series ID": 0, "Catalogue ID": 0, "Use Unit Dims": True,
        "UDX": 0, "UDY": 0, "UDZ": 0, "Use Unit Cursor Moves": True,
        "PREDISTX:": 0, "PREMETHODX:": "SpecifiedDist",
        "PREDISTY:": 0, "PREMETHODY:": "SpecifiedDist",
        "PREDISTZ:": 0, "PREMETHODZ:": "SpecifiedDist", "PREROT:": 0,
        "POSTDISTX:": 0, "POSTMETHOSX:": "UnitDistPos",
        "POSTDISTY:": 0, "POSTMETHODSTY:": "SpecifiedDist",
        "POSTDISTZ:": 0, "POSTMETHODZ:": "SpecifiedDist", "POSTROT": 0,
        "Clash With Parent": False, "Handing": "None", "Status": "Compulsory",
        "Lock Linked Unit Attributes": True
    }

    for col_data in parsed_collections:
        c_id = col_data["id"]
        c_name = col_data["name"]
        units = col_data["units"]

        for i, u_code in enumerate(units):
            row = static_vals.copy()
            row["Link List ID"] = c_id
            row["Link List Name"] = c_name
            row["Unit Number"] = i + 1
            row["UnitName"] = u_code
            tab2_rows.append(row)

    df2_final = pd.DataFrame(tab2_rows)

    # Ordered Headers
    tab2_cols = [
        "Add OR Delete", "Link List ID", "Link List Name", "Unit Number", "UnitName",
        "Series ID", "Catalogue ID", "Use Unit Dims", "UDX", "UDY", "UDZ",
        "Use Unit Cursor Moves", "PREDISTX:", "PREMETHODX:", "PREDISTY:",
        "PREMETHODY:", "PREDISTZ:", "PREMETHODZ:", "PREROT:", "POSTDISTX:",
        "POSTMETHOSX:", "POSTDISTY:", "POSTMETHODSTY:", "POSTDISTZ:",
        "POSTMETHODZ:", "POSTROT", "Clash With Parent", "Handing",
        "Status", "Lock Linked Unit Attributes"
    ]
    for c in tab2_cols:
        if c not in df2_final.columns: df2_final[c] = ""
    df2_final = df2_final[tab2_cols]

    # --- PART E: DOWNLOAD ---
    base_name = os.path.splitext(filename)[0]
    file1_name = f"{base_name}_CopyLinklist.xlsx"
    file2_name = f"{base_name}_Import_Linklist.xlsx"

    print(f"✅ Saving {file1_name}...")
    df1_final.to_excel(file1_name, index=False)

    print(f"✅ Saving {file2_name}...")
    df2_final.to_excel(file2_name, index=False)

    print(f"\nDownloading both files...")
    files.download(file1_name)
    files.download(file2_name)

    print(f"Done! (Total time: {round(time.time() - start_time, 2)}s)")